In [1]:
from helper import Helper
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.optimizers import Adam, RMSprop, SGD
from tensorflow.keras.layers import GlobalAveragePooling2D, Dropout, Dense, Flatten
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping, ModelCheckpoint
from time import time
import matplotlib.pyplot as plt

In [2]:
h = Helper()

train_it, validation_it, test_it = h.construct_data_generator_w_validation(batch_size=10)

Found 22050 images belonging to 3 classes.
Found 3150 images belonging to 3 classes.
Found 6300 images belonging to 3 classes.


In [3]:
mobilenet = MobileNet(include_top=False, weights=None, input_shape=(224,224,3))
mobilenet.summary()

Model: "mobilenet_1.00_224"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNormaliza (None, 112, 112, 32

In [4]:
x=mobilenet.output
x=GlobalAveragePooling2D()(x)
x=Dense(1024,activation='relu')(x) 
x=Dense(1024,activation='relu')(x)
x=Dense(512,activation='relu')(x)
preds=Dense(3,activation='softmax')(x)

model = Model(inputs=mobilenet.input, outputs=preds, name='cnn_topomap_classification_model_mobilenet')
model.summary()

Model: "cnn_topomap_classification_model_mobilenet"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
conv1_pad (ZeroPadding2D)    (None, 225, 225, 3)       0         
_________________________________________________________________
conv1 (Conv2D)               (None, 112, 112, 32)      864       
_________________________________________________________________
conv1_bn (BatchNormalization (None, 112, 112, 32)      128       
_________________________________________________________________
conv1_relu (ReLU)            (None, 112, 112, 32)      0         
_________________________________________________________________
conv_dw_1 (DepthwiseConv2D)  (None, 112, 112, 32)      288       
_________________________________________________________________
conv_dw_1_bn (BatchNorma

In [5]:
opt = Adam(learning_rate=0.0001)
loss = 'categorical_crossentropy'

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [6]:
tblog_path = h.logs_directory.format(time())
tb_log = TensorBoard(log_dir = tblog_path, profile_batch=0)

ch = ModelCheckpoint('models/mobilenet_checkpoints/cnn_topomap_classification_model_mobilenet.h5', monitor='val_accuracy', 
                     mode='max', save_best_only=True)

# es = EarlyStopping(monitor='val_loss', mode='min', min_delta=0.0001, patience=10)

In [7]:
r = model.fit_generator(train_it, epochs=100, callbacks=[ch, tb_log], validation_data=validation_it)

Epoch 1/100
2205/2205 [==============================] - 249s 113ms/step - loss: 0.9754 - accuracy: 0.4991 - val_loss: 1.5951 - val_accuracy: 0.3765
Epoch 2/100
2205/2205 [==============================] - 249s 113ms/step - loss: 0.7363 - accuracy: 0.6786 - val_loss: 1.6042 - val_accuracy: 0.3854
Epoch 3/100
2205/2205 [==============================] - 250s 114ms/step - loss: 0.6308 - accuracy: 0.7355 - val_loss: 0.7217 - val_accuracy: 0.6956
Epoch 4/100
2205/2205 [==============================] - 250s 113ms/step - loss: 0.5563 - accuracy: 0.7668 - val_loss: 0.8915 - val_accuracy: 0.6197
Epoch 5/100
2205/2205 [==============================] - 249s 113ms/step - loss: 0.5029 - accuracy: 0.7942 - val_loss: 0.7397 - val_accuracy: 0.6835
Epoch 6/100
2205/2205 [==============================] - 250s 113ms/step - loss: 0.4599 - accuracy: 0.8119 - val_loss: 0.8356 - val_accuracy: 0.6613
Epoch 7/100
2205/2205 [==============================] - 250s 113ms/step - loss: 0.4282 - accuracy: 0.8264

Epoch 56/100
2205/2205 [==============================] - 248s 112ms/step - loss: 0.0671 - accuracy: 0.9771 - val_loss: 0.7465 - val_accuracy: 0.7505
Epoch 57/100
2205/2205 [==============================] - 248s 113ms/step - loss: 0.0625 - accuracy: 0.9766 - val_loss: 0.6272 - val_accuracy: 0.8340
Epoch 58/100
2205/2205 [==============================] - 248s 112ms/step - loss: 0.0626 - accuracy: 0.9780 - val_loss: 0.8219 - val_accuracy: 0.7940
Epoch 59/100
2205/2205 [==============================] - 248s 113ms/step - loss: 0.0600 - accuracy: 0.9796 - val_loss: 0.8347 - val_accuracy: 0.7587
Epoch 60/100
2205/2205 [==============================] - 249s 113ms/step - loss: 0.0635 - accuracy: 0.9776 - val_loss: 0.3935 - val_accuracy: 0.8790
Epoch 61/100
2205/2205 [==============================] - 247s 112ms/step - loss: 0.0592 - accuracy: 0.9788 - val_loss: 1.1080 - val_accuracy: 0.7213
Epoch 62/100
2205/2205 [==============================] - 248s 113ms/step - loss: 0.0555 - accuracy:

In [5]:
model_ch = h.load('models/mobilenet_checkpoints/cnn_topomap_classification_model_mobilenet.h5')
e_ch = model_ch.evaluate_generator(test_it)
e_ch

[0.3819703352594355, 0.89301586]

In [6]:
E = model.evaluate_generator(test_it)
E

NameError: name 'model' is not defined